In [ ]:
#Веса модели и различные сопустствующие функции можно взять из репо ниже
#!git clone https://github.com/paveldat/object_detection_on_video.git

# #Установим необходимые библиотеки
# !pip install art

In [1]:
!pip install opencv-python
!pip install art
!pip install pandas
!pip install matplotlib

!pip install pyTelegramBotAPI

In [1]:
#Библиотеки
import cv2
import numpy as np
import pandas as pd
import pandas.io.sql as sqlio
from art import tprint
import matplotlib.pylab as plt
import requests
import telebot
from telebot import types
import datetime
import time
import os
import psycopg2 as ps




In [2]:
TOKEN = "7639010467:AAG4D-qM36c4MPqlPKkCM_I6ACZNEHqZ9Os"
bot=telebot.TeleBot(TOKEN)



In [3]:
conn2 = ps.connect(dbname = "postgres",#park_BD#postgres
                    user = "postgres",
                    password = "1",
                    host = "localhost",
                    port = "5432")
#connection.autocommit = True

In [4]:
##### Пути до конфигурации и весов модели
time.sleep(12)
path_conf = "./yolov4-tiny.cfg"
path_weights = "./yolov4-tiny.weights"

#Классы которые распознает YOLO
path_coco_names = "./coco.names.txt"

#Тестовое видео
# video_path = './final_test.MOV'
#video_path = './part_1.mp4'
#video_path = './playlist.mp4'
#video_path = './test_output/output.mp4'
#video_path = './Видосы_с_камеры/output.mp4'
video_path = './Видосы_с_камеры/output_1.mp4'
otpr = './Видосы_с_камеры/output_1.mp4' 
video_path_1 = './Видосы_с_камеры/output_1.mp4'
video_path_2 = './Видосы_с_камеры/output_2.mp4'



In [5]:

#Функции для подсчета ResNet
def calculate_iou(box, boxes, box_area, boxes_area):
    #Считаем IoU
    y1 = np.maximum(box[0], boxes[:, 0])
    y2 = np.minimum(box[2]+box[0], boxes[:, 2]+boxes[:, 0])
    x1 = np.maximum(box[1], boxes[:, 1])
    x2 = np.minimum(box[3]+box[1], boxes[:, 3]+boxes[:, 1])
    intersection = np.maximum(x2 - x1, 0) * np.maximum(y2 - y1, 0)
    union = box_area + boxes_area[:] - intersection[:]
    iou = intersection / union
    return iou

#Функция для расчета персечения всех со всеми через IoU
def compute_overlaps(boxes1, boxes2):
    #Areas of anchors and GT boxes
    area1 = boxes1[:, 2] * boxes1[:, 3]
    area2 = boxes2[:, 2] * boxes2[:, 3]
    overlaps = np.zeros((boxes1.shape[0], boxes2.shape[0]))
    for i in range(overlaps.shape[1]):
        box2 = boxes2[i]
        overlaps[:, i] = calculate_iou(box2, boxes1, area2[i], area1)
    return overlaps

#Функция для отрисовки Bounding Box в кадре
def draw_bbox(x, y, w, h, parking_text, parking_color = (0, 255, 0)):

    start = (x, y)
    end = (x + w, y + h)
    color = parking_color
    width = 2
    final_image = cv2.rectangle(image_to_process, start, end, color, width)

    #Подпись BB
    start = (x, y - 10)
    font_size = 0.4
    font = cv2.FONT_HERSHEY_SIMPLEX
    width = 1
    text = parking_text
    final_image = cv2.putText(final_image, text, start, font, font_size, color, width, cv2.LINE_AA)
    return final_image

def draw_bbox2(x, y, w, h, parking_text, parking_color = (0, 255, 0)):

    start = (x, y)
    end = (x + w, y + h)
    color = parking_color
    width = 2
    final_image_pust = cv2.rectangle(image_to_process, start, end, color, width)

    return final_image_pust


#Функция для отправки фото в телеграм
def send_photo_file(chat_id, img):
    files = {'photo': open(img, 'rb')}
    requests.post(f'https://api.telegram.org/bot{TOKEN}/sendPhoto?chat_id={chat_id}', files=files)

#Функция для отправки сообщения в телеграм
def send_telegram_message(message,chat_id):
    requests.get(f'https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={chat_id}&text={message}').json()




In [6]:
def proverka(connection,final_image):
    connection.autocommit = True
    cursor = connection.cursor()

    select_users = """SELECT * FROM table_1"""
    cursor.execute(select_users)
    results = cursor.fetchall()
    
    df = pd.DataFrame(results)
    df.columns = ['ID','Chat_ID','Onlines','Zapros_Foto']
    #return df
    count_banana_all = (df['Zapros_Foto'] == True).sum()
    if count_banana_all > 0:
        otpravlenie_video = """SELECT "Chat_ID" FROM table_1 WHERE "Zapros_Foto" = True LIMIT 1"""
        cursor.execute(otpravlenie_video)
        results = cursor.fetchall()
        df = pd.DataFrame(results)
        df.columns = ['ID']
        chat_id = df.loc[0,'ID']
        #os.remove(r'D:\UOD\image_test_not_free.png')
        cv2.imwrite('./image_test_not_free.png', final_image)
        send_photo_file(chat_id, './image_test_not_free.png')
        os.remove(r'D:\UOD\image_test_not_free.png')
        otpravlenie_video = (f"""UPDATE table_1 SET "Zapros_Foto" = False WHERE "Chat_ID" ={chat_id};""")
        cursor.execute(otpravlenie_video)
        


In [7]:
def prover_v_sety(novost,connection,final_image):
    connection.autocommit = True
    cursor = connection.cursor()
    
    select_users = """SELECT * FROM table_1"""
    cursor.execute(select_users)
    results = cursor.fetchall()
    
    df = pd.DataFrame(results)
    df.columns = ['ID','Chat_ID','Onlines','Zapros_Foto']
    #return df
    count_banana_all = (df['Onlines'] == True).sum()
    if count_banana_all > 0:
        print(count_banana_all)
        if novost == 1:
            otpravlenie_video = """SELECT "Chat_ID" FROM table_1 WHERE "Onlines" = True"""
            cursor.execute(otpravlenie_video)
            results = cursor.fetchall()
            df = pd.DataFrame(results)
            df.columns = ['ID']
            #return df
            for i in range(count_banana_all):
                chat_id = df.loc[i,'ID']
                message_tel = 'Где ты ездишь??? Место уже занято :('
                send_telegram_message(message_tel,chat_id)
                cv2.imwrite('./image_test_free.png', final_image)
                send_photo_file(chat_id, './image_test_free.png')
                os.remove(r'D:\UOD\image_test_free.png')
                otpravlenie_video = (f"""UPDATE table_1 SET "Onlines" = False WHERE "Chat_ID" ={chat_id};""")
                cursor.execute(otpravlenie_video)
                
        else:
            otpravlenie_video = """SELECT "Chat_ID" FROM table_1 WHERE "Onlines" = True"""
            cursor.execute(otpravlenie_video)
            results = cursor.fetchall()
            df = pd.DataFrame(results)
            df.columns = ['ID']
            for i in range(count_banana_all):

                chat_id = df.loc[i,'ID']
                message_tel = 'Свободное место! Давай, жми скорее!!!'
                send_telegram_message(message_tel,chat_id)
                
                cv2.imwrite('./image_test_free.png', final_image)
                send_photo_file(chat_id, './image_test_free.png')
                os.remove(r'D:\UOD\image_test_free.png')
                otpravlenie_video = (f"""UPDATE table_1 SET "Onlines" = False WHERE "Chat_ID" ={chat_id};""")
                cursor.execute(otpravlenie_video)
        for i in range(count_banana_all):
            chat_id = df.loc[i,'ID']
            otpravlenie_video = (f"""UPDATE table_1 SET "Onlines" = True WHERE "Chat_ID" ={chat_id};""")
            cursor.execute(otpravlenie_video)           
                

In [9]:
#Определяем параметры модели
#Загружаем конфигурацию и веса модели скаченные ранее
net = cv2.dnn.readNetFromDarknet(path_conf, path_weights)
layer_names = net.getLayerNames()
out_layers_indexes = net.getUnconnectedOutLayers()
out_layers = [layer_names[index - 1] for index in out_layers_indexes]

#Парковочные места
first_frame_parking_spaces = None

free_parking_timer = 0
free_parking_timer_bag1 = 0
free_parking_count = 0
first_parking_timer = 0
free_parking_space = False
free_parking_space_box = None
check_det_frame = None
m = 0

chat_id = ""

#Сообщение в телеграм?
telegram_message = False

#Инициализируем работу с видео
video_capture = cv2.VideoCapture(video_path_1)

nomer = 0 
frame_counter = 0

#Пока не нажата клавиша q функция будет работать
while video_capture.isOpened():

    m = m + 1 #Счетчик прохождений цикла
    frame_counter += 1
    
    ret, image_to_process = video_capture.read()

    #Препроцессинг изображения и работа YOLO
    height, width, _ = image_to_process.shape
    blob = cv2.dnn.blobFromImage(image_to_process, 1 / 255, (608, 608),
                                 (0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(out_layers)
    class_indexes, class_scores, boxes = ([] for i in range(3))

    #Обнаружение объектов в кадре
    for out in outs:
        for obj in out:
            scores = obj[5:]
            class_index = np.argmax(scores)

            #В классе 2 (car) только автомобили
            if class_index == 2:
                class_score = scores[class_index]
                if class_score > 0:
                    center_x = int(obj[0] * width)
                    center_y = int(obj[1] * height)
                    obj_width = int(obj[2] * width)
                    obj_height = int(obj[3] * height)
                    box = [center_x - obj_width // 2, center_y - obj_height // 2,
                            obj_width, obj_height]

                    #BBoxes
                    boxes.append(box)
                    class_indexes.append(class_index)
                    class_scores.append(float(class_score))


    ###ПЕРВЫЙ КАДР ОПРЕДЕЛЯЕМ ПАРКОМЕСТА
    
    if (m == 1) and (not first_frame_parking_spaces):
        #Предполагаем, что под каждой машиной будет парковочное место
        first_frame_parking_spaces = boxes
        first_frame_parking_score = class_scores

    else:
        chosen_cars_boxes = cv2.dnn.NMSBoxes(boxes, class_scores, 0.0, 0.4)
        cars_area = []

        ###МАШИНЫ
        for box_index in chosen_cars_boxes:
            car_box = boxes[box_index]
            cars_area.append(car_box)

            x, y, w, h = car_box
            parking_text = 'Car'
            final_image = draw_bbox(x, y, w, h, parking_text, (255, 255, 0))
            final_image_pust = draw_bbox2(x, y, w, h, parking_text, (255, 255, 0))

        #Теперь зная парковочные места, определим когда место освободится
        cars_boxes = cars_area

        ###IoU
        #if m % 20 == 0:
        overlaps = compute_overlaps(np.array(parking_spaces), np.array(cars_boxes))

        for parking_space_one, area_overlap in zip(parking_spaces, overlaps):

            max_IoU = max(area_overlap)
            sort_IoU = np.sort(area_overlap[area_overlap > 0])[::-1]

            if free_parking_space == False:

                if 0.0 < max_IoU < 0.4:

                    #Количество паркомест по условию 1: 0.0 < IoU < 0.4
                    len_sort = len(sort_IoU)

                    #Количество паркомест по условию 2: IoU > 0.15
                    sort_IoU_2 = sort_IoU[sort_IoU > 0.15]
                    len_sort_2 = len(sort_IoU_2)

                    #Смотрим чтобы удовлятворяло условию 1 и условию 2
                    if (check_det_frame == parking_space_one) & (len_sort != len_sort_2):
                        #Начинаем считать кадры подряд с пустыми координатами
                        free_parking_timer += 1

                    elif check_det_frame == None:
                        check_det_frame = parking_space_one

                    else:
                        #Фильтр от чехарды мест (если место чередуется, то "скачет")
                        free_parking_timer_bag1 += 1
                        if free_parking_timer_bag1 == 2:
                            #Обнуляем счётчик, если паркоместо "скачет"
                            check_det_frame = parking_space_one
                            free_parking_timer = 0

                    #Если более 10 кадров подряд, то предполагаем, что место свободно
                    if free_parking_timer == 10:
                        #Помечаем свободное место
                        free_parking_space = True
                        free_parking_space_box = parking_space_one
                        #Отрисовываем рамку парковочного места
                        x_free, y_free, w_free, h_free = parking_space_one

            else:
                #Если место занимают, то помечается как отсутствие свободных мест
                overlaps = compute_overlaps(np.array([free_parking_space_box]), np.array(cars_boxes))
                for area_overlap in overlaps:
                    max_IoU = max(area_overlap)
                    if max_IoU > 0.6:

                        free_parking_space = False
                        telegram_message = False

                        #Отправка сообщения боту в телеграмм
                        if not telegram_message:
                            screenshot_parking_space = final_image
                            #отправим в телеграм
                            prover_v_sety(1,conn2,final_image)


                            telegram_message = True


    ###ПАРКОВОЧНЫЕ МЕСТА
    #Отрисовка BB парковочных мест
    chosen_boxes = cv2.dnn.NMSBoxes(first_frame_parking_spaces,
                                    first_frame_parking_score, 0.0, 0.4)
    parking_spaces = []

    for box_index in chosen_boxes:
        box = first_frame_parking_spaces[box_index]

        #Если определилось пустое место, то отрисуем его в кадре
        if free_parking_space:
            if box == [x_free, y_free, w_free, h_free]:
                parking_text = 'FREE SPACE!!!'
                final_image = draw_bbox(x_free, y_free, w_free, h_free, parking_text, (0, 0, 255))
            else:
                x, y, w, h = box
                parking_text = 'No parking'
                final_image = draw_bbox(x, y, w, h, parking_text)

                #Отправка сообщения боту в телеграмм
                if not telegram_message:
                #Скриншот свободного места, отправим в телеграм
                    screenshot_parking_space = final_image
                
                    prover_v_sety(0,conn2,final_image)

                    telegram_message = True

        else:
            #Координаты и размеры BB
            x, y, w, h = box
            parking_text = 'No parking'
            final_image = draw_bbox(x, y, w, h, parking_text)

        #Координаты парковочных мест с первого кадры
        parking_spaces.append(box)
    #Показать результат работы
    cv2.imshow("Parking Space", final_image)
    #cv2.imshow("Parking Space", final_image_pust)
    
    if frame_counter == int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT)):
        nomer = nomer + 1
        frame_counter = 0
        if(nomer % 2 == 0 ):
            video_capture.release()
            cv2.destroyAllWindows()
            
            video_path_2 = './Видосы_с_камеры/output_2.mp4'
            video_capture = cv2.VideoCapture(video_path_2)
        else:
            video_capture.release()
            cv2.destroyAllWindows()
            video_path_1 = './Видосы_с_камеры/output_1.mp4'
            video_capture = cv2.VideoCapture(video_path_1)

   
    proverka(conn2,final_image)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        #Очищаем всё после завершения.
        video_capture.release()
        cv2.destroyAllWindows()
        break
        



2
2
2
2
2
2
1
1


In [7]:
final_image = './Видосы_с_камеры/image_test_free.png'
prover_v_sety(0,conn2,final_image)

NameError: name 'prover_v_sety' is not defined

In [38]:
final_image = './Видосы_с_камеры/image_test_not_free.png'
proverka(conn2,final_image)

error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'imwrite'
> Overload resolution failed:
>  - img is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'img'


In [5]:
def foto(final_image,message):
    cv2.imwrite('./image_test_not_free.png', final_image)
    chat_id = message.from_user.id
    send_photo_file(chat_id, './image_test_not_free.png')
    time.sleep(5)
    os.remove(r'D:\UOD\image_test_not_free.png')

In [2]:
def obnav():
    video_path = './test_output/output.mp4'
    video_capture = cv2.VideoCapture(video_path)

In [13]:
#Проверка работоспособности бота
@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
	bot.reply_to(message, "Howdy, how are you doing?")

@bot.message_handler(func=lambda message: True)
def echo_all(message):
	bot.reply_to(message, message.text)

bot.infinity_polling()

2024-10-22 20:08:39,567 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"


2024-10-13 21:45:36,993 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"


In [24]:
@bot.message_handler(commands=['start'])
def start_message(message):
    your_variable = message.from_user.id
    print(your_variable)
    bot.send_message(message.chat.id, 'test')
bot.infinity_polling()

2024-10-13 21:47:22,518 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"


In [ ]:
#Определяем параметры модели
#Загружаем конфигурацию и веса модели скаченные ранее
net = cv2.dnn.readNetFromDarknet(path_conf, path_weights)
layer_names = net.getLayerNames()
out_layers_indexes = net.getUnconnectedOutLayers()
out_layers = [layer_names[index - 1] for index in out_layers_indexes]

#Парковочные места
first_frame_parking_spaces = None

free_parking_timer = 0
free_parking_timer_bag1 = 0
free_parking_count = 0
first_parking_timer = 0
free_parking_space = False
free_parking_space_box = None
check_det_frame = None
m = 0

#Сообщение в телеграм?
telegram_message = False

#Инициализируем работу с видео
video_capture = cv2.VideoCapture(video_path)

#Пока не нажата клавиша q функция будет работать
while video_capture.isOpened():

    m = m + 1 #Счетчик прохождений цикла

    ret, image_to_process = video_capture.read()

    #Препроцессинг изображения и работа YOLO
    height, width, _ = image_to_process.shape
    blob = cv2.dnn.blobFromImage(image_to_process, 1 / 255, (608, 608),
                                 (0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(out_layers)
    class_indexes, class_scores, boxes = ([] for i in range(3))

    #Обнаружение объектов в кадре
    for out in outs:
        for obj in out:
            scores = obj[5:]
            class_index = np.argmax(scores)

            #В классе 2 (car) только автомобили
            if class_index == 2:
                class_score = scores[class_index]
                if class_score > 0:
                    center_x = int(obj[0] * width)
                    center_y = int(obj[1] * height)
                    obj_width = int(obj[2] * width)
                    obj_height = int(obj[3] * height)
                    box = [center_x - obj_width // 2, center_y - obj_height // 2,
                            obj_width, obj_height]

                    #BBoxes
                    boxes.append(box)
                    class_indexes.append(class_index)
                    class_scores.append(float(class_score))


    ###ПЕРВЫЙ КАДР ОПРЕДЕЛЯЕМ ПАРКОМЕСТА
    
    if (m == 1) and (not first_frame_parking_spaces):
        #Предполагаем, что под каждой машиной будет парковочное место
        first_frame_parking_spaces = boxes
        first_frame_parking_score = class_scores

    else:
        chosen_cars_boxes = cv2.dnn.NMSBoxes(boxes, class_scores, 0.0, 0.4)
        cars_area = []

        ###МАШИНЫ
        for box_index in chosen_cars_boxes:
            car_box = boxes[box_index]
            cars_area.append(car_box)

            x, y, w, h = car_box
            parking_text = 'Car'
            final_image = draw_bbox(x, y, w, h, parking_text, (255, 255, 0))
            final_image_pust = draw_bbox2(x, y, w, h, parking_text, (255, 255, 0))

        #Теперь зная парковочные места, определим когда место освободится
        cars_boxes = cars_area

        ###IoU
        overlaps = compute_overlaps(np.array(parking_spaces), np.array(cars_boxes))

        for parking_space_one, area_overlap in zip(parking_spaces, overlaps):

            max_IoU = max(area_overlap)
            sort_IoU = np.sort(area_overlap[area_overlap > 0])[::-1]

            if free_parking_space == False:

                if 0.0 < max_IoU < 0.4:

                    #Количество паркомест по условию 1: 0.0 < IoU < 0.4
                    len_sort = len(sort_IoU)

                    #Количество паркомест по условию 2: IoU > 0.15
                    sort_IoU_2 = sort_IoU[sort_IoU > 0.15]
                    len_sort_2 = len(sort_IoU_2)

                    #Смотрим чтобы удовлятворяло условию 1 и условию 2
                    if (check_det_frame == parking_space_one) & (len_sort != len_sort_2):
                        #Начинаем считать кадры подряд с пустыми координатами
                        free_parking_timer += 1

                    elif check_det_frame == None:
                        check_det_frame = parking_space_one

                    else:
                        #Фильтр от чехарды мест (если место чередуется, то "скачет")
                        free_parking_timer_bag1 += 1
                        if free_parking_timer_bag1 == 2:
                            #Обнуляем счётчик, если паркоместо "скачет"
                            check_det_frame = parking_space_one
                            free_parking_timer = 0

                    #Если более 10 кадров подряд, то предполагаем, что место свободно
                    if free_parking_timer == 10:
                        #Помечаем свободное место
                        free_parking_space = True
                        free_parking_space_box = parking_space_one
                        #Отрисовываем рамку парковочного места
                        x_free, y_free, w_free, h_free = parking_space_one

            else:
                #Если место занимают, то помечается как отсутствие свободных мест
                overlaps = compute_overlaps(np.array([free_parking_space_box]), np.array(cars_boxes))
                for area_overlap in overlaps:
                    max_IoU = max(area_overlap)
                    if max_IoU > 0.6:

                        free_parking_space = False
                        telegram_message = False

                        #Отправка сообщения боту в телеграмм
                        if not telegram_message:
                            screenshot_parking_space = final_image
                            #отправим в телеграм
                            message_tel = 'Где ты ездишь??? Место уже занято :('
                            send_telegram_message(message_tel)
                            cv2.imwrite('./image_test_not_free.png', screenshot_parking_space)
                            send_photo_file(chat_id, './image_test_not_free.png')

                            telegram_message = True


    ###ПАРКОВОЧНЫЕ МЕСТА
    #Отрисовка BB парковочных мест
    chosen_boxes = cv2.dnn.NMSBoxes(first_frame_parking_spaces,
                                    first_frame_parking_score, 0.0, 0.4)
    parking_spaces = []

    for box_index in chosen_boxes:
        box = first_frame_parking_spaces[box_index]

        #Если определилось пустое место, то отрисуем его в кадре
        if free_parking_space:
            if box == [x_free, y_free, w_free, h_free]:
                parking_text = 'FREE SPACE!!!'
                final_image = draw_bbox(x_free, y_free, w_free, h_free, parking_text, (0, 0, 255))
            else:
                x, y, w, h = box
                parking_text = 'No parking'
                final_image = draw_bbox(x, y, w, h, parking_text)

            #Отправка сообщения боту в телеграмм
            if not telegram_message:
                #Скриншот свободного места, отправим в телеграм
                screenshot_parking_space = final_image
                message_tel = 'Свободное место! Давай, жми скорее!!!'
                send_telegram_message(message_tel)
                cv2.imwrite('./image_test_free.png', screenshot_parking_space)
                send_photo_file(chat_id, './image_test_free.png')

                telegram_message = True

        else:
            #Координаты и размеры BB
            x, y, w, h = box
            parking_text = 'No parking'
            final_image = draw_bbox(x, y, w, h, parking_text)

        #Координаты парковочных мест с первого кадры
        parking_spaces.append(box)
    #Показать результат работы
    cv2.imshow("Parking Space", final_image)
    #cv2.imshow("Parking Space", final_image_pust)

    #Прерывание работы клавишей q
    #if ret:
    #    #cv2.imshow('Video', frame)
   #     if cv2.waitKey(1) & 0xFF == ord('q'):
  #          video_capture.release()
 #           cv2.destroyAllWindows()
#            break
   # else:
    #    if(video_path == "./output1.mp4" ):
    #        video_path = "./output2.mp4"
   #     else:
  #          video_path = "./output1.mp4"
 #       video_capture = cv2.VideoCapture(video_path)
#        ret, image_to_process = video_capture.read()
    if cv2.waitKey(1) & 0xFF == ord('q'):
        #Очищаем всё после завершения.
        video_capture.release()
        cv2.destroyAllWindows()
        break

    
    #@bot.message_handler(commands=['button'])
    #def button_message(message):
    #	markup=types.ReplyKeyboardMarkup(resize_keyboard=True)
    #    item1=types.KeyboardButton("Кнопка")
    #    markup.add(item1)
    #    bot.reply_to(message, "Howdy, how are you doing?")
    #bot.infinity_polling()
   # @bot.message_handler(commands=['button'])
   # def button_message(message):
   #     markup=types.ReplyKeyboardMarkup(resize_keyboard=True)
   #     item1=types.KeyboardButton("Кнопка")
   #     markup.add(item1)
   #     bot.send_message(message.chat.id,'Выберите что вам надо',reply_markup=markup)
   # @bot.message_handler(content_types='text')
   # def message_reply(message):
   #     if message.text=="Кнопка":
   #         bot.send_message(message.chat.id,"https://habr.com/ru/users/lubaznatel/")
   # bot.infinity_polling()
    #if m == 15:
    #    obnav()
    # Каждые +- 13 проходов проходит 15 секунд
    # Новое видео с камеры получается +- 2-3 секунды(звисит ои скорости интернета)
    # На 9-10 проходе надо запускать скрипт по записи нового видоса
    # на 12-13 проходе надо запускать видос под номером 1, после прохождения этого цикла запускать снова первый фрагмент который будет перезаписан
